# Project
## Include Libraries and initial settings
### Import all the libraries

In [28]:
# Import all the libraries
import pandas as pd
import plotly.express as px
from src.utils.plots import *
import numpy as np

# Set some internal settings for plotly
px.defaults.width = 800
px.defaults.height = 600
px.defaults.template = 'plotly_white'

# Define the folder
FOLDER = 'data/processed/'
SAVING_FOLDER = 'docs/plots/'

### Load and filter the data
#### Load the data

In [29]:
# Load the data
df_beers = pd.read_parquet(FOLDER + 'beers.pq')
df_breweries = pd.read_parquet(FOLDER + 'breweries.pq')
df_users = pd.read_parquet(FOLDER + 'users.pq')
df_ratings_no_text = pd.read_parquet(FOLDER + 'ratings_no_text.pq')

#### Filter the data

In [30]:
df_ratings_no_text = df_ratings_no_text[df_ratings_no_text['date'].dt.year>=2002]

## Data presentation
Let's take a look at the data we have in the dataset.

In [31]:
df_beers.head(5)

,beer_id,beer_name,brewery_id,brewery_name,style,abv
0,410549,33 Export (Gabon),3198,Sobraga,Pale Lager,5.0
1,105273,Castel Beer (Gabon),3198,Sobraga,Pale Lager,5.2
2,19445,Régab,3198,Sobraga,Pale Lager,4.5
3,155699,Ards Bally Black Stout,13538,Ards Brewing Co.,Stout,4.6
4,239097,Ards Belfast 366,13538,Ards Brewing Co.,Golden Ale/Blond Ale,4.2


In [32]:
print(f"In the platform there are {df_beers.shape[0]} different beers")

In the platform there are 399987 different beers


In [33]:
df_breweries.head(5)

,brewery_id,brewery_name,country_brewery,state_brewery
0,3198,Sobraga,Gabon,None
1,13538,Ards Brewing Co.,United Kingdom,None
2,22304,Barrahooley Craft Brewery,United Kingdom,None
3,22818,Boundary,United Kingdom,None
4,24297,Brewbot Belfast,United Kingdom,None


In [34]:
print(f"In the platform there are {df_breweries.shape[0]} different breweries")

In the platform there are 24189 different breweries


In [35]:
df_users.head(5)

,user_id,user_name,joined,country_user,state_user
0,175852,Manslow,2012-05-20 10:00:00,Poland,None
1,442761,MAGICuenca91,2017-01-10 11:00:00,Spain,None
2,288889,Sibarh,2013-11-16 11:00:00,Poland,None
3,250510,fombe89,2013-03-22 11:00:00,Spain,None
4,122778,kevnic2008,2011-02-02 11:00:00,Germany,None


In [36]:
print(f"In the platform there are {df_users.shape[0]} different users")

In the platform there are 50592 different users


In [37]:
df_ratings_no_text.head(5)

,date,beer_id,user_id,brewery_id,abv,style,rating,palate,taste,appearance,aroma,overall,year,brewery_name,country_brewery,state_brewery,country_user,state_user
0,2016-04-26 12:00:00,410549,175852,3198,5.0,Pale Lager,2.0,2.0,4.0,2.0,4.0,8.0,2016,Sobraga,Gabon,None,Poland,None
1,2017-02-17 12:00:00,105273,442761,3198,5.2,Pale Lager,1.9,2.0,4.0,2.0,3.0,8.0,2017,Sobraga,Gabon,None,Spain,None
2,2016-06-24 12:00:00,105273,288889,3198,5.2,Pale Lager,1.6,2.0,3.0,3.0,3.0,5.0,2016,Sobraga,Gabon,None,Poland,None
3,2016-01-01 12:00:00,105273,250510,3198,5.2,Pale Lager,1.5,1.0,2.0,4.0,3.0,5.0,2016,Sobraga,Gabon,None,Spain,None
4,2015-10-23 12:00:00,105273,122778,3198,5.2,Pale Lager,1.9,2.0,4.0,2.0,4.0,7.0,2015,Sobraga,Gabon,None,Germany,None


In [38]:
print(f"In the platform there are {df_ratings_no_text.shape[0]} different ratings")
print(f"The first rating was made on {df_ratings_no_text['date'].min()}")
print(f"The last rating was made on {df_ratings_no_text['date'].max()}")

In the platform there are 7102520 different ratings
The first rating was made on 2002-01-01 12:00:00
The last rating was made on 2017-07-31 12:00:00


In [39]:
df_beers = df_beers.merge(df_breweries[['brewery_id', 'country_brewery', 'state_brewery']], on='brewery_id', how='left').rename(columns={'country_brewery': 'country_beer', 'state_brewery': 'state_beer'})
df_beers.head(5)

,beer_id,beer_name,brewery_id,brewery_name,style,abv,country_beer,state_beer
0,410549,33 Export (Gabon),3198,Sobraga,Pale Lager,5.0,Gabon,None
1,105273,Castel Beer (Gabon),3198,Sobraga,Pale Lager,5.2,Gabon,None
2,19445,Régab,3198,Sobraga,Pale Lager,4.5,Gabon,None
3,155699,Ards Bally Black Stout,13538,Ards Brewing Co.,Stout,4.6,United Kingdom,None
4,239097,Ards Belfast 366,13538,Ards Brewing Co.,Golden Ale/Blond Ale,4.2,United Kingdom,None


Now that we know what's in our data, let's see how our data is distributed over space.

In [40]:
# Compute the aggregated value for countries outside the US
beers_per_country_no_US = df_beers[df_beers['country_beer'] != 'United States'].groupby('country_beer').size().reset_index(name='count').rename(columns={'country_beer': 'location'})
users_per_country_no_US = df_users[df_users['country_user'] != 'United States'].groupby('country_user').size().reset_index(name='count').rename(columns={'country_user': 'location'})
breweries_per_country_no_US = df_breweries[df_breweries['country_brewery'] != 'United States'].groupby('country_brewery').size().reset_index(name='count').rename(columns={'country_brewery': 'location'})
ratings_in_country_no_US = df_ratings_no_text[df_ratings_no_text['country_brewery'] != 'United States'].groupby('country_brewery').size().reset_index(name='count').rename(columns={'country_brewery': 'location'})
ratings_users_country_no_US = df_ratings_no_text[df_ratings_no_text['country_user'] != 'United States'].groupby('country_user').size().reset_index(name='count').rename(columns={'country_user': 'location'})
df_no_US = [beers_per_country_no_US, users_per_country_no_US, breweries_per_country_no_US, ratings_in_country_no_US, ratings_users_country_no_US]

# Compute the aggregated value for the states of the US
beers_per_country_US = df_beers[df_beers['country_beer'] == 'United States'].groupby('state_beer').size().reset_index(name='count').rename(columns={'state_beer': 'location'})
users_per_country_US = df_users[df_users['country_user'] == 'United States'].groupby('state_user').size().reset_index(name='count').rename(columns={'state_user': 'location'})
breweries_per_country_US = df_breweries[df_breweries['country_brewery'] == 'United States'].groupby('state_brewery').size().reset_index(name='count').rename(columns={'state_brewery': 'location'})
ratings_in_country_US = df_ratings_no_text[df_ratings_no_text['country_brewery'] == 'United States'].groupby('state_brewery').size().reset_index(name='count').rename(columns={'state_brewery': 'location'})
ratings_users_country_US = df_ratings_no_text[df_ratings_no_text['country_user'] == 'United States'].groupby('state_user').size().reset_index(name='count').rename(columns={'state_user': 'location'})
df_US = [beers_per_country_US, users_per_country_US, breweries_per_country_US, ratings_in_country_US, ratings_users_country_US]

# Define some options for the plot
options = {
    "title": "Beer Statistics by Country and US State",
    "plots": [
        {
            'label': 'Beers per country',
            'location_label': 'location',
            'z_label': 'count',
            'colorscale': 'Blues'
        },{
            'label': 'Users per country',
            'location_label': 'location',
            'z_label': 'count',
            'colorscale': 'Blues'
        }, {
            'label': 'Breweries per country',
            'location_label': 'location',
            'z_label': 'count',
            'colorscale': 'Blues'
        }, {
            'label': 'Number of ratings based on the brewery country',
            'location_label': 'location',
            'z_label': 'count',
            'colorscale': 'Blues'
        }, {
            'label': 'Number of ratings based on the reviewer country',
            'location_label': 'location',
            'z_label': 'count',
            'colorscale': 'Blues'            
        }
    ]
}

# Plot the map
plot_map(df_no_US, df_US, options)

Finally just review the temporal evolution of our data.

In [41]:
year_grouping = df_ratings_no_text.groupby(df_ratings_no_text['date'].dt.year).size().reset_index(name='count').rename(columns={'date': 'Year', 'count': 'Number of ratings'})
px.bar(year_grouping, x='Year', y='Number of ratings', title=f'Number of ratings per year').show()

In [42]:
year_grouping = df_users.groupby(df_users['joined'].dt.year).size().reset_index(name='count').rename(columns={'joined': 'Year', 'count': 'Number of users'})
px.bar(year_grouping, x='Year', y='Number of users', title=f'Number of users that has joined each year').show()

## Preliminary preference analysis
In this section we are going to analyze the preferences of the users of the platform in a general way.
### Correlation between the ratings attributes
Here we are gonna analyze the correlation between the ratings attributes given by the users. 

In [43]:
corr = df_ratings_no_text[['palate','taste','appearance','aroma','overall','abv']].corr()
plot_correlation_matrix(df_ratings_no_text[['palate','taste','appearance','aroma','overall','abv']], filename=SAVING_FOLDER + 'correlation_matrix.html', title='Correlation matrix between the ratings provided by the users')

All factors influence the overall user experience, but taste (correlation: 0.86) and aroma (correlation: 0.77) stand out as the most significant. Palate (correlation: 0.66) and appearance (correlation: 0.50) show slightly lower correlations with overall experience, though their contribute remain important.

### Rating in the different countries

In [44]:
# Remove the countries with less than 250 ratings
number_of_ratings_per_country = df_ratings_no_text.groupby('country_brewery').size().reset_index().rename(columns={0:'count'})
number_of_ratings_per_country = number_of_ratings_per_country[number_of_ratings_per_country['count'] > 250]
df_ratings_filtered = df_ratings_no_text[df_ratings_no_text['country_brewery'].isin(number_of_ratings_per_country['country_brewery'])]

# Compute the average rating per country
average_rating_no_US = df_ratings_filtered[df_ratings_filtered['country_brewery'] != 'United States'].groupby('country_brewery')['rating'].mean().reset_index().rename(columns={'country_brewery':'location', 'rating':'count'})
average_rating_US = df_ratings_filtered[df_ratings_filtered['country_brewery'] == 'United States'].groupby('state_brewery')['rating'].mean().reset_index().rename(columns={'state_brewery':'location', 'rating':'count'})

# Plot everything
options = {
    "title": "Average Ratings by Country and US State",
    "plots": [{
        'label': 'Beers per country',
        'location_label': 'location',
        'z_label': 'count',
        'colorscale': 'Blues'
    }]
}
plot_map(average_rating_no_US, average_rating_US, options)

In [45]:
average_rating = pd.concat([average_rating_no_US, average_rating_US]).sort_values('count', ascending=False)
average_rating.head(10)

,location,count
35,Oklahoma,3.647186
1,Alaska,3.628913
4,California,3.607066
34,Ohio,3.558747
12,Illinois,3.550528
36,Oregon,3.535853
47,Washington DC,3.516449
10,Belgium,3.504538
44,Vermont,3.502204
9,Georgia,3.494033


In [46]:
average_rating = pd.concat([average_rating_no_US, average_rating_US]).sort_values('count', ascending=True)
average_rating.head(10)

,location,count
52,Iran,1.499602
31,El Salvador,1.847122
85,Nicaragua,1.847436
44,Guatemala,1.858252
24,Cuba,1.929189
113,Uganda,1.933131
30,Egypt,1.937105
117,Venezuela,1.939168
28,Dominican Republic,1.982480
108,Tanzania,2.003830


### Rating evolution over time in the different countries

In [47]:
# Create the variables
row_US = []
row_no_US = []

# Prepare the data
for year in df_ratings_no_text['date'].dt.year.unique():
    df_state_no_US = df_ratings_no_text[(df_ratings_no_text['date'].dt.year == year) & (df_ratings_no_text['country_brewery'] != 'United States')].groupby('country_brewery').agg({'rating': 'mean'}).reset_index()
    row_no_US += [{'year': year, 'location': state, 'count': abv} for state, abv in zip(df_state_no_US['country_brewery'], df_state_no_US['rating'])]

    df_state_US = df_ratings_no_text[(df_ratings_no_text['date'].dt.year == year) & (df_ratings_no_text['country_brewery'] == 'United States')].groupby('state_brewery').agg({'rating': 'mean'}).reset_index()
    row_US += [{'year': year, 'location': state, 'count': abv} for state, abv in zip(df_state_US['state_brewery'], df_state_US['rating'])]

# Create the dataframes
df_states_no_US = pd.DataFrame(row_no_US)
df_states_US = pd.DataFrame(row_US)

# Filter the data for a more clean plot (No US)
number_of_years_per_state_no_US = df_states_no_US.groupby('location').size().reset_index().rename(columns={0:'count'})
number_of_years_per_state_no_US = number_of_years_per_state_no_US[number_of_years_per_state_no_US['count'] == len(df_ratings_no_text['date'].dt.year.unique())]
df_states_no_US = df_states_no_US[df_states_no_US['location'].isin(number_of_years_per_state_no_US['location'])]

# Filter the data for a more clean plot (US)
number_of_years_per_state_US = df_states_US.groupby('location').size().reset_index().rename(columns={0:'count'})
number_of_years_per_state_US = number_of_years_per_state_US[number_of_years_per_state_US['count'] == len(df_ratings_no_text['date'].dt.year.unique())]
df_states_US = df_states_US[df_states_US['location'].isin(number_of_years_per_state_US['location'])]

# Define the options for the plot
options = {
    'title': '',
    'time_range': range(df_ratings_no_text['date'].dt.year.min(), df_ratings_no_text['date'].dt.year.max() + 1),
    'time_label': 'year',
    'location_label': 'location',
    'value_label': 'count',
    'range_color': [1, 4],
    'color_scale': 'Viridis'
}

# Display the plot
plot_map_time(df_states_no_US, df_states_US, options)

## ABV and style analysis
In this section we'll continue by analyzing the popularity of ABV and style in the world and we'll also study how these have evolved over time.

In [48]:
# Define some useful variables
MAX_ABV = 20
NUMBER_OF_SAMPLES_ABV = 201
MIN_NUMBER_OF_RATING = 250

# Process the data
beer_ratings = []
linspace = np.linspace(0, MAX_ABV, NUMBER_OF_SAMPLES_ABV)
for year in sorted(df_ratings_no_text['date'].dt.year.unique()):
    # Filter the data by the year
    df_year = df_ratings_no_text[df_ratings_no_text['year'] == year]
    
    # Iterate within the ABV range
    for i in range(len(linspace) - 1):
        # Filter the data
        min_abv = round(linspace[i], 2)
        max_abv = round(linspace[i + 1], 2)

        # Compute the matrics
        filtered = df_year[(df_year['abv'] >= min_abv) & (df_year['abv'] < max_abv)]
        ratings = filtered['rating'].mean()
        nbr_ratings = filtered['rating'].count()

        # Append the data
        if nbr_ratings > MIN_NUMBER_OF_RATING:
            beer_ratings.append({'year': year, 'abv': (min_abv+max_abv)/2, 'rating': ratings, 'nbr_ratings': nbr_ratings})

# Convert to DataFrame
beer_ratings = pd.DataFrame(beer_ratings)

# Do the plot
fig = px.scatter(beer_ratings, x='abv', y='rating', size='nbr_ratings', hover_name='abv',animation_frame='year', labels={'abv': 'ABV:', 'rating': 'Rating:', 'nbr_ratings': 'Number of ratings:'},range_x=[0, 20], range_y=[2.25, 4.75])
fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
fig.update_layout(showlegend=False)
fig.update_xaxes(title_text='ABV')
fig.update_yaxes(title_text='Rating')
fig.show()

In [ ]:
# Compute the metrics
avg_abv = df_ratings_no_text.groupby('year').agg({'abv': 'mean'}).reset_index()
corr = avg_abv['year'].corr(avg_abv['abv'], method='spearman')

# Plot the graph
fig = px.line(avg_abv, x='year', y='abv', title='Average ABV Over Years')
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Average ABV',
    xaxis=dict(tickvals=np.arange(2002, 2018, 1)),
    yaxis=dict(range=[5, 7]),
    showlegend=False
)
fig.add_annotation(
    x=2010,
    y=7,
    text=f'<b>Spearman correlation: {corr:.2f}</b>',
    showarrow=False,
    font=dict(size=14)
)
fig.show()

## Brewery popularity analysis
In this section we are going to see whether user likes a lot beers from specific breweries from specific countries and in general we'll focus on understanding the impact of breweries on the popularity of beers.

## NLP analysis
In this final section we are going to see from the textual ratings if there are some specific words or emotions that are associated with high or low ratings.